In [1]:
# Imports Required Libraries
import asyncio
import os
import pandas as pd
import pickle
import statsmodels.api as sm
import time

# Imported in order of function sequence
from common import transform_json_to_df
from common import batch_df

# Synchronous Method Versions
from common import extract_transform_predict_df_batches

# Asynchronous Method Versions
from common import async_extract_transform_predict_df_batches

# May aid with the Pickle file loading, functions however without, better safe than sorry
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
def load_models():
    '''
    Returns loaded in, pre-trained Prediction, SimpleImputer and StandardScalar models
    '''
    
    mdl = sm.load('../models/glm_final_model.pickle')
    si = pickle.load(open('../models/glm_simple_imputer.pickle', 'rb'))
    ss = pickle.load(open('../models/glm_standard_scalar.pickle', 'rb'))
    
    return mdl, si, ss

In [3]:
def sync_main(rows_per_batch, input_json):
    '''
    Returns a single event or a list of events as a JSON message containing the business outcome, 
        probability of said outcome, along with the input variables which led to said outcome 
        in alphanumerical order for all those predictions which met the minimum standard of 75% 
        chance of a successful sale to a potential buying customer
    
    Keyword Arguments:
    input_json -- Raw JSON data
    '''
    
    # List of the already properly ordered column variables required by the pre-trained
    #    model in order for it to carry out accurate predictions
    final_df_column_variable_names_order = [
        'x5_saturday', 'x81_July', 'x81_December', 'x31_japan', 'x81_October', 
        'x5_sunday', 'x31_asia', 'x81_February', 'x91', 'x81_May', 
        'x5_monday', 'x81_September', 'x81_March', 'x53', 'x81_November', 
        'x44', 'x81_June', 'x12', 'x5_tuesday', 'x81_August', 
        'x81_January', 'x62', 'x31_germany', 'x58', 'x56']
    
    alphanumerically_sorted_df_column_variable_names = sorted(final_df_column_variable_names_order)
    
    mdl, si, ss = load_models()
    
    df = transform_json_to_df(input_json)
    
    df_batches = batch_df(rows_per_batch, df)
    
    starting_time = time.time()
    
    json_output_message = extract_transform_predict_df_batches(df_batches, 
                                                               si, 
                                                               ss, 
                                                               final_df_column_variable_names_order, 
                                                               mdl, 
                                                               alphanumerically_sorted_df_column_variable_names)
    
    print(f'Time To Completion: {time.time() - starting_time} Seconds \n')

    return json_output_message

In [4]:
async def async_main(rows_per_batch, input_json):
    '''
    Returns a single event or a list of events as a JSON message containing the business outcome, 
        probability of said outcome, along with the input variables which led to said outcome 
        in alphanumerical order for all those predictions which met the minimum standard of 75% 
        chance of a successful sale to a potential buying customer
    
    Keyword Arguments:
    input_json -- Raw JSON data
    '''
    
    # List of the already properly ordered column variables required by the pre-trained
    #    model in order for it to carry out accurate predictions
    final_df_column_variable_names_order = [
        'x5_saturday', 'x81_July', 'x81_December', 'x31_japan', 'x81_October', 
        'x5_sunday', 'x31_asia', 'x81_February', 'x91', 'x81_May', 
        'x5_monday', 'x81_September', 'x81_March', 'x53', 'x81_November', 
        'x44', 'x81_June', 'x12', 'x5_tuesday', 'x81_August', 
        'x81_January', 'x62', 'x31_germany', 'x58', 'x56']
    
    alphanumerically_sorted_df_column_variable_names = sorted(final_df_column_variable_names_order)
    
    mdl, si, ss = load_models()
    
    df = transform_json_to_df(input_json)
    
    df_batches = batch_df(rows_per_batch, df)
    
    starting_time = time.time()
    
    json_output_message = await async_extract_transform_predict_df_batches(
        df_batches, si, ss, final_df_column_variable_names_order, 
        mdl, alphanumerically_sorted_df_column_variable_names)
    
    print(f'Time To Completion: {time.time() - starting_time} Seconds \n')

    return json_output_message

In [5]:
# For Debugging & Testing Purposes
raw_testing_data1 = pd.read_csv(os.path.join('../testing', 'exercise_26_test.csv'))

raw_testing_data2 = pd.read_csv(os.path.join('../testing', 'exercise_26_test.csv'), nrows = 1)
raw_testing_data3 = pd.read_csv(os.path.join('../testing', 'exercise_26_test.csv'), nrows = 10)
raw_testing_data4 = pd.read_csv(os.path.join('../testing', 'exercise_26_test.csv'), nrows = 100)
raw_testing_data5 = pd.read_csv(os.path.join('../testing', 'exercise_26_test.csv'), nrows = 1000)

#raw_testing_data1.head()
#raw_testing_data2.head()
#raw_testing_data3.head()
#raw_testing_data4.head()
#raw_testing_data5.head()

In [6]:
sample_raw_json_10000_rows = raw_testing_data1.to_json(orient = 'records')

sample_raw_json_1_row_v2 = raw_testing_data2.to_json(orient = 'records')
sample_raw_json_10_rows = raw_testing_data3.to_json(orient = 'records')
sample_raw_json_100_rows = raw_testing_data4.to_json(orient = 'records')
sample_raw_json_1000_rows = raw_testing_data5.to_json(orient = 'records')

In [7]:
sample_raw_json_1_row_v1 = "{\"x0\":0.042317,\"x1\":-3.344721,\"x2\":4.6351242122,\"x3\":-0.5983959993,\"x4\":-0.6477715046,\"x5\":\"monday\",\"x6\":0.184902,\"x7\":46.690015,\"x8\":3.034132,\"x9\":0.364704,\"x10\":14.260733,\"x11\":-1.559332,\"x12\":\"$5,547.78\",\"x13\":0.520324,\"x14\":31.212255,\"x15\":4.891671,\"x16\":0.357763,\"x17\":14.766366,\"x18\":-17.467243,\"x19\":0.224628,\"x20\":0.096752,\"x21\":1.305564,\"x22\":0.353632,\"x23\":3.909028,\"x24\":-91.273052,\"x25\":1.396952,\"x26\":4.401593,\"x27\":0.443086,\"x28\":14.048787,\"x29\":-0.932243,\"x30\":5.255472,\"x31\":\"germany\",\"x32\":0.54199153,\"x33\":2.98948039,\"x34\":-1.78334189,\"x35\":0.80127315,\"x36\":-2.60231221,\"x37\":3.39682926,\"x38\":-1.22322646,\"x39\":-2.20977636,\"x40\":-68.69,\"x41\":522.25,\"x42\":-428.69,\"x43\":381.37,\"x44\":0.0197503,\"x45\":0.75116479,\"x46\":0.8630479008,\"x47\":-1.0383166613,\"x48\":-0.2726187635,\"x49\":-0.3430207259,\"x50\":0.3109008666,\"x51\":-0.797841974,\"x52\":-2.0390175153,\"x53\":0.87182889,\"x54\":0.14373012,\"x55\":-1.15212514,\"x56\":-2.1703139704,\"x57\":-0.267842962,\"x58\":0.212110633,\"x59\":1.6926559407,\"x60\":-0.9522767913,\"x61\":-0.8625864974,\"x62\":0.0748487158,\"x63\":\"36.29%\",\"x64\":3.47125327,\"x65\":-3.16656509,\"x66\":0.65446814,\"x67\":14.60067029,\"x68\":-20.57521013,\"x69\":0.71083785,\"x70\":0.16983767,\"x71\":0.55082127,\"x72\":0.62814576,\"x73\":3.38608078,\"x74\":-112.45263714,\"x75\":1.48370808,\"x76\":1.77035368,\"x77\":0.75702363,\"x78\":14.75731742,\"x79\":-0.62550355,\"x80\":null,\"x81\":\"October\",\"x82\":\"Female\",\"x83\":-0.7116680715,\"x84\":-0.2653559892,\"x85\":0.5175495907,\"x86\":-1.0881027092,\"x87\":-1.8188638198,\"x88\":-1.3584469527,\"x89\":-0.654995195,\"x90\":-0.4933042262,\"x91\":0.373853,\"x92\":0.94143481,\"x93\":3.54679834,\"x94\":-99.8574882,\"x95\":0.403926,\"x96\":1.65378726,\"x97\":0.00771459,\"x98\":-32.02164582,\"x99\":-60.3127828}"

In [8]:
# Saves raw JSON data files for proper testing when Docker becomes involved
#with open(os.path.join('../testing', 'sample_raw_json_1_row_v1.json'), 'w') as file:
#    file.write(sample_raw_json_1_row_v1)

In [9]:
# Retrieves slices of raw testing data of various sizes whilst 
#    converting and saving them into a JSON format
#sample_raw_json_10000_rows = raw_testing_data1.to_json(os.path.join('../testing', 'sample_raw_json_10000_rows.json'), orient = 'records')

#sample_raw_json_1_row_v2 = raw_testing_data2.to_json(os.path.join('../testing', 'sample_raw_json_1_row_v2.json'), orient = 'records')
#sample_raw_json_10_rows = raw_testing_data3.to_json(os.path.join('../testing', 'sample_raw_json_10_rows.json'), orient = 'records')
#sample_raw_json_100_rows = raw_testing_data4.to_json(os.path.join('../testing', 'sample_raw_json_100_rows.json'), orient = 'records')
#sample_raw_json_1000_rows = raw_testing_data5.to_json(os.path.join('../testing', 'sample_raw_json_1000_rows.json'), orient = 'records')

In [10]:
def results_details(json_res, print_res):
    print(f'Results Type: {type(json_res)}\n')
    print(f'Results Length: {len(json_res)}\n')
    
    if isinstance(json_res, list) and print_res == True:
        for result in json_res:
            print(f'{result}\n')

In [11]:
# Results Length: Expected = 0
json_results1 = sync_main(500, sample_raw_json_1_row_v1)
results_details(json_results1, True)

Time To Completion: 0.014991283416748047 Seconds 

Results Type: <class 'list'>

Results Length: 0



In [12]:
# Results Length: Expected = 0
json_results2 = sync_main(500, sample_raw_json_1_row_v2)
results_details(json_results2, True)

Time To Completion: 0.014630317687988281 Seconds 

Results Type: <class 'list'>

Results Length: 0



In [13]:
# Results Length: Expected = 2
json_results3 = sync_main(500, sample_raw_json_10_rows)
results_details(json_results3, True)

Time To Completion: 0.01764988899230957 Seconds 

Results Type: <class 'list'>

Results Length: 2

{'business_outcome': '1', 'p_hat': '0.8227526744741207', 'x12': -0.9529553121992512, 'x31_asia': 0.0, 'x31_germany': 0.0, 'x31_japan': 0.0, 'x44': -0.018906143399875407, 'x53': -1.5974242627501094, 'x56': 1.6483959900371876, 'x58': 0.599511719639024, 'x5_monday': 0.0, 'x5_saturday': 0.0, 'x5_sunday': 0.0, 'x5_tuesday': 1.0, 'x62': 0.7274434380391581, 'x81_August': 0.0, 'x81_December': 0.0, 'x81_February': 0.0, 'x81_January': 0.0, 'x81_July': 0.0, 'x81_June': 0.0, 'x81_March': 0.0, 'x81_May': 0.0, 'x81_November': 1.0, 'x81_October': 0.0, 'x81_September': 0.0, 'x91': -0.3633580994460479}

{'business_outcome': '1', 'p_hat': '0.7539305190612656', 'x12': 1.1180442609776802, 'x31_asia': 0.0, 'x31_germany': 1.0, 'x31_japan': 0.0, 'x44': 1.0491037054747567, 'x53': -1.0452443300416803, 'x56': -0.9224261424370558, 'x58': 0.41258044127697047, 'x5_monday': 0.0, 'x5_saturday': 0.0, 'x5_sunday': 0.0, '

In [14]:
# Results Length: Expected = 24
json_results4 = sync_main(500, sample_raw_json_100_rows)
results_details(json_results4, False)

Time To Completion: 0.03506207466125488 Seconds 

Results Type: <class 'list'>

Results Length: 24



In [15]:
# Results Length: Expected = 215
json_results5 = sync_main(500, sample_raw_json_1000_rows)
results_details(json_results5, False)

Time To Completion: 0.29364824295043945 Seconds 

Results Type: <class 'list'>

Results Length: 215



In [16]:
# Results Length: Expected = 2013
json_results6 = sync_main(100, sample_raw_json_10000_rows)
results_details(json_results6, False)

Time To Completion: 3.046194076538086 Seconds 

Results Type: <class 'list'>

Results Length: 2013



In [17]:
# Results Length: Expected = 2013
json_results7 = await async_main(100, sample_raw_json_10000_rows)
results_details(json_results7, False)

Time To Completion: 3.0248148441314697 Seconds 

Results Type: <class 'list'>

Results Length: 2013

